<center><br><br>
    TANF Data Collaborative <br>
    Applied Data Analytics Training | Spring 2022
    <h1> Creating a Cohort </h1>
    <span style="font-size: 1.5em;">
        <a href='https://www.coleridgeinitiative.org'>Coleridge Initiative</a>
    </span>
    <center>Benjamin Feder, Maryah Garner, Allison Nunez, Carolyn D. Gorman</center>
</center>

***

Now that you have been introduced to the datasets available for you in this training and have taken a sufficient amount of time to explore potentially relevant variables, you should be ready to begin mapping out the beginnings of your longitudinal analysis. This notebook introduces you to the concept of creating a group or a "cohort" that defines the primary population of interest in much research. A cohort is a group of individuals that share a time-based characteristic. For the purposes of this training program, your cohort will consist of a population of TANF recipients. Once created, cohorts may then be used to link to other data sources, such as employment data.

As we create a cohort in this notebook, we will walk you through some of the decisions that need to be made when defining a population of TANF recipients; such as filters to define exiters, the amount of exits, age, and work-eligibility. Once this is complete, we will construct descriptive statistics to better understand the population of interest.

## 1. Getting started

In [ ]:
options(warn = -1)                   # switches warnings off

suppressMessages(library(odbc))      # allows R to connect with the database
suppressMessages(library(tidyverse)) # useful for data manipulation and visualization
suppressMessages(library(scales))    # to calculate percentages, graphing
suppressMessages(library(lubridate)) # for easy working with dates 

options(warn = 0)                    # switches warnings on 
options(scipen=999)                  # prevents scientific notation

In [ ]:
# server connection
con <- DBI::dbConnect(odbc::odbc(),
                     Driver = "SQL Server",
                     Server = "msssql01.c7bdq4o2yhxo.us-gov-west-1.rds.amazonaws.com",
                     Trusted_Connection = "True")

## 2. Defining a Cohort
 In this notebook, we will explore a cohort of individuals who exited TANF during the second quarter of 2018. In your own projects, you might want to explore alternative constructions; such as individuals who entered TANF during a particular time window. 
 
> Note: We use the condition WHERE **tanf_end = 1** to select only observations that are the last record of a spell, or span of time. 

We will start our cohort exploration by defining this basic analytical frame from the complete **person_month_cleaned** table in the **tr_tdc_2022** database. It may take a few seconds to read the data. 

> Note: It is common practice to only include the necessary variables within your cohort definition. You can narrow down the list of variables through a combination of an exploratory data analysis and familiarizing yourself with the data dictionary. Also note that we are using the complete **person_month_cleaned** table because we are only selecting individuals who exited TANF during the second quarter of 2018. Please do not try to use the entire **person_month_cleaned** table without selecting out a small subset of individuals.* 

In [ ]:
# Create query character string
    # Database name: tr_tdc_2022
    # Schema name: dbo
    # Table name: person_month_clean

query <- "
SELECT ssn, caseid, dob, yr_month, welig, affil, tanf_total_months, tanf_start, hispan, 
    native, asian, black, hawaia, white, county
FROM tr_tdc_2022.dbo.person_month_clean
WHERE yr_quarter = '2018 Q2'  AND tanf_end = 1
"

tanf_cohort <- dbGetQuery(con, query)

# view the first 6 observations
head(tanf_cohort)

With our 10% sample, we saw that there were instances where the same individual would show up multiple times in the data. Let's see if this also occurs when we focus on a time-bound group of exiters.

In [ ]:
# find number of rows, unique people, and unique cases in TANF data
tanf_cohort %>%
    summarize(
        n_rows = n(),
        n_cases = n_distinct(caseid),
        n_people = n_distinct(ssn)
    )

Indeed, this does appear to be the case, as evidenced by the difference between the number of rows and the number of people.

### Potential Cohort Filters

The previous analysis indicates that there are some people that show up more than once with multiple TANF exits within the same quarter in our base cohort (exiters in **2018 Q2**). Additionally, based on our work in the Exploratory Data Analysis notebook, we expect there to be some minors in our cohort. For this analysis, we may want to filter out such individuals from our final analytical cohort.

In this section, we will explore a handful of potential cohort filters. As you work through this section, we encourage you to think about other filters you may want to explore when developing your own cohort.

#### Multiple Exits

Duplicates are often present in administrative data, especially when working with data that result from merging two or more data sets together. Regardless of your population of interest, you may be working with sets of duplicate rows on your unit of interest (**ssn**). Depending on the analysis, you may not want to double-count individuals that satisfy the cohort constraint (TANF exits within a quarter). Instead, in this notebook, we may want to focus only on their most recent occurrence (exit).

Let's look at an instance where an individual appears multiple times in the data.

In [ ]:
# find example of someone showing up multiple times
# use pull to isolate the contents of the variable (individual ssn) and save as an object
ssn_dup <- tanf_cohort %>%
    count(ssn) %>%
    arrange(desc(n)) %>%
    head(1) %>% 
    pull(ssn)

tanf_cohort %>%
    filter(ssn == ssn_dup)

You can see that the **tanf_total_months** variable updates between their exits. Let's limit our cohort to only instances of a last TANF exit within this time frame.

In [ ]:
# select last exit (yr_month) for each ssn
tanf_cohort_clean <- tanf_cohort %>%
    arrange(ssn, desc(yr_month)) %>%
    distinct(ssn, .keep_all = T) 

In [ ]:
# confirm we have one row per ssn
tanf_cohort_clean %>%
    summarize(
        n_rows = n(),
        n_people = n_distinct(ssn)
    )

By applying this restriction, we now have a unique sample of observations.

#### Age

Next, we will use the code we developed in the EDA notebook to manually create our **age** variable. In this notebook, we will additionally standardize the age across the multiple months of exit (second quarter) by finding their age at the end of the quarter.

In [ ]:
# calculate age_end
tanf_cohort_clean <- tanf_cohort_clean %>%  
    mutate(
        age_end = trunc((dob %--% ymd('2018/06/30')) / years(1)) # find age at end of Q2 2018
    ) 

# see age_end as the last variable
head(tanf_cohort_clean)

Let's see if the data frame contains minors.

In [ ]:
# count number of minors and non-minors
tanf_cohort_clean %>%
    mutate(
        adult_ind = ifelse(age_end >= 18, "Adult", "Not_Adult")
    ) %>%
    count(adult_ind)

Because minors are less likely work-eligibile, with the context of focusing on employment outcomes, we will filter out all individuals who were not 18 at the end of 2018 Q2.

In [ ]:
# filter for 18+
tanf_cohort_clean <- tanf_cohort_clean %>%
    filter(age_end >= 18)

# confirm filter worked as intended-n_people should be the same as the number of adults found above
tanf_cohort_clean %>%
    summarize(
        n_people = n_distinct(ssn)
    )

#### Work-Eligibility Status

The variable **welig**, the work-eligible individual indicator, will help us to delve further into the work-eligibility statuses of these individuals.

In [ ]:
# see breakdown of individuals by welig
# note: can use count() because there is one row per ssn, otherwise would use summarize(n_distinct(ssn))
tanf_cohort_clean %>%
    count(welig)

We *highly* encourage you to explore the data dictionary for this variable, as there may be certain values you want to exclude from your analytical frame. The **welig** value of 12, for example, indicates that the individual has passed away. Additionally, according to the data dictionary, all **welig** values greater than 5 stipulate that the person was not eligible for work.

For the sake of the exercise, we will filter out all individuals who were not eligible for work. 

In [ ]:
# filter all non-work eligible individuals
tanf_cohort_clean <- tanf_cohort_clean %>%
    filter(welig %in% c("01", "02", "03", "04", "05"))

# see number of individuals
tanf_cohort_clean %>%
    summarize(
        n_people = n_distinct(ssn)
    )

#### Family Affiliation

To narrow the cohort definition even further, we can focus on the **affil** variable, which tracks the affiliation of the individual to the one receiving assistance.

In [ ]:
# see distribution of affil
tanf_cohort_clean %>%
    count(affil)

We can see that a portion of the current sample includes **affil** values of 2, which refers to a parent of the minor child in the eligible family receiving assistance. If you wanted to only concentrate on the member of the family receiving assistance, as is the case for this example, you would want to filter for only individuals with **affil** values equal to 1.

> Note: While we are exploring the employment outcomes of TANF recipients in this notebook, there is also strong interest in understanding the employment outcomes of individuals who are parents, guardians, or caretakers of minors who are TANF recipients. You may want to explore this on your own.

In [ ]:
# filter for only affil = 1
tanf_cohort_clean <- tanf_cohort_clean %>%
    filter(affil == 1)

tanf_cohort_clean %>% 
    summarize(
        n_people = n_distinct(ssn)
    )

Now that we have settled on a specific data-driven cohort definition, we will explore our cohort to get a better sense of our analytical frame.

## 3. Understanding the Cohort

After creating your cohort, you may want to transition right into record linkage and begin your longitudinal analysis. While this is understandable, it is important to take the time to learn more about your cohort. This step will provide you with additional verification of the cohort construction and allow you to contextualize future findings. As with EDA, we strongly encourage you to examine your analytical frame and discuss any missingness and abnormal values.

In this notebook, we will calculate descriptive statistics on the following:
- Time on TANF
- Race/Ethnicity

As you're working through this section, think about possible subgroup and measurement distributions you may want to implement in your project.

### Time Receiving TANF

Because our research question focuses on a group of individuals exiting a TANF spell, we will take a look at the distribution of the **tanf_total_months** variable.

In [ ]:
# visual summary of total time on tanf
tanf_cohort_clean %>%
    ggplot(aes(x=tanf_total_months)) + 
    geom_histogram()

Recall that this variable is right-censored (values will not be higher than a certain amount) because all TANF cases were recorded as new cases in the data if they existed in the TANF system in the first month of the data. If this censoring had not occured, such individuals would likely show up in the data with even longer TANF spells.

Instead of treating **tanf_total_months** as a numeric variable, we may want to recode the column to group lengths of TANF stays in order to classify our cohort into potential subgroups. The **tanf_length_cat** variable was constructed following the guidance of our TDC pilot teams. 

In [ ]:
# create a categorical variable based on tanf_total_months
tanf_cohort_clean <- tanf_cohort_clean %>%
    mutate(
        tanf_length_cat = case_when(
            tanf_total_months <= 6 ~ "short",
            tanf_total_months > 6 & tanf_total_months <= 24 ~ "medium", 
            tanf_total_months > 24 ~ "long",
            TRUE ~ "error"
        )
    ) 
    
# see distribution of stays
tanf_cohort_clean %>%
    count(tanf_length_cat)

By analyzing the spell variable in a categorical fashion, we can see that there are far fewer individuals with long TANF stays. 

> Note: In the next notebook, after linking the cohort to Indiana wage records, we will examine employment outcomes for those in the cohort based on their relative length of times receiving TANF benefits (**tanf_length_cat**).

### Race/Ethnicity Distribution

An important demographic breakdown worth considering in your analytical framework is the race/ethnicity distribution of the cohort. There are a multitude of race- and ethnicity-related variables in the TANF data. By examining the set of columns in **tanf_cohort_clean** (and **person_month_clean**), we can see just how many variables there are.

In [ ]:
# see all of the different race-related variables (focus on variables from hispan to white)
names(tanf_cohort_clean)

Upon inspection, there appear to be six related variables. We could look at the distribution of the six variables grouped together:

In [ ]:
# example of doing all in one big table--it's hard to read!! and doesn't meet disclosure thresholds!!
tanf_cohort_clean %>%
    count(hispan, native, asian, black, hawaia, white)

Instead, we can combine these variables into an all-encompassing one. Due to the small counts within the combination of these six variables, we will opt to recode them as an indicator that an individual's race/ethnicity is white, non-hispanic:

In [ ]:
# instead can recode race/ethnicity to get only white indicator
tanf_cohort_clean %>%
    mutate(
        white_only_ind = ifelse(white == '1' & hispan == '2' & native == '2' & asian == '2' & black == '2' & hawaia == '2' , "white only", "not white only")
    ) %>%
    count(white_only_ind)

We can see that the majority of individuals in the cohort are not non-hispanic white.

## 4. Saving as a Permanent Table

Since we performed the cohort filtering in R, we can save the final cohort definition as a csv file. We can also save the file as a permanent table in R, which will allow us to leverage SQL's join functionality to link the cohort to large tables that are unable to be read into R due to memory constraints. The following code blocks demonstrate how to convert an R data frame to a permanent table by creating the table **nb_cohort** in the TDC training workspace database **tr_tdc_2022** from the data frame **tanf_cohort_clean**.

> Note: Saving your cohort as a permanent table is *highly* recommended. When doing so, please save your table with the prefix **team#** (ex. **team1_cohort**) to differentiate between tables for each project.

    qry <- "use tr_tdc_2022;"
    DBI::dbExecute(con, qry)

    DBI::dbWriteTable(
        conn = con,
        name = DBI::SQL("dbo.nb_cohort"), 
        value = tanf_cohort_clean
    )

> Note: If you run this code, you will get an error because a table with the name `nb_cohort` already exists in the `tr_tdc_2022` database.

## 5. Teaser: Linking Different Cohorts

Given the ultimate focus of tracking employment outcomes of TANF exiters, we can justify some of our cohort creation decisions. As an extra step, we can compare the linkage rate to the wage records in the following year after TANF exit based on the loose cohort construction presented at the beginning of the notebook compared to the one we eventually narrowed down to (`tanf_cohort_clean`).

> Note: We will cover the programmatic steps involved in the linkage in the following notebook. 

In [ ]:
# linkage with initial cohort definition of all TANF exiters during the second quarter of 2018
# also includes potential duplicates
qry <- "
select *
from tr_tdc_2022.dbo.example_cohort_link
"
df_initial <- dbGetQuery(con, qry)

initial_wages <- df_initial %>%
    summarize(
        n_individuals = n_distinct(ssn)
    )

initial_wages

In [ ]:
# linkage with refined cohort definition (tanf_cohort_clean)
qry <- "
select *
from tr_tdc_2022.dbo.nb_cohort_link
"
df_refined <- dbGetQuery(con, qry)

refined_wages <- df_refined %>%
    summarize(
        n_individuals = n_distinct(ssn)
    )

refined_wages

In [ ]:
# see percentage of people with employment records in at least one quarter in the following year
initial_result <- tanf_cohort %>%
    summarize(
        cohort_size = n_distinct(ssn),
        cohort_w_wages = initial_wages$n_individuals,
        match_prop = initial_wages$n_individuals/cohort_size
    ) %>%
    mutate(
        cohort_definition = "initial"
    )

initial_result

In [ ]:
# see percentage of people with employment records in at least one quarter in the following year
cohort_result <- tanf_cohort_clean %>%
    summarize(
        cohort_size = n_distinct(ssn),
        cohort_w_wages = refined_wages$n_individuals,
        match_prop = refined_wages$n_individuals/cohort_size
    ) %>%
    mutate(
        cohort_definition = "refined"
    )

cohort_result

In [ ]:
# see match rate in a bar plot
# rbind() combines the two data frames as long as they have the same column names and types
initial_result %>%
    rbind(cohort_result) %>%
    ggplot(aes(x=cohort_definition, y = match_prop)) +
    geom_col()

Despite narrowing the cohort size by nearly REDACTED, we only lose out on employment records for less than REDACTED individuals while increasing the linkage by almost REDACTED. Due to our explicit focus on employment outcomes, we can further justify the filtering process in our cohort creation to more directly involve only those individuals likely to be employed. 

## References
Cumpton, Greg, Feder, Benjamin, Barrett, Nathan, & Mian, Rukhshan. (2022, April 4). Data Exploration for Cohort Analysis using Texas Higher Education Coordinating Board Data. Zenodo. https://doi.org/10.5281/zenodo.6412617